In [5]:
import os

import contextily as ctx
import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from matplotlib import cm
import networkx as nx
from joblib import Parallel, delayed

In [6]:
enriched_points = dd.read_csv(
    "./anglova_metrics_enriched/points_enriched_metrics.csv",
).compute()
enriched_points["t"] = pd.to_datetime(enriched_points["t"])
enriched_trajectories_df = pd.read_csv(
    "./anglova_metrics_enriched/trajectories_enriched_metrics.csv",
)
pair_wise_distances = dd.read_csv(
    "./pairwise_distances.csv",
).compute()


MemoryError: Unable to allocate 729. MiB for an array with shape (95518800,) and data type int64

In [ ]:
output_path = "./assets/Leader Following Patterns/"

In [ ]:
def save_plot(filename):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    plt.savefig(
        f"{output_path}/{filename}.png",
        bbox_inches="tight",
    )
    plt.savefig(
        f"{output_path}/{filename}.pdf",
        bbox_inches="tight",
    )

In [ ]:
cud_okabe_ito_colors = [
    "#E69F00",  # orange
    "#56B4E9",  # sky blue
    "#009E73",  # green
    "#F0E442",  # yellow
    "#0072B2",  # blue
    "#D55E00",  # reddish orange
    "#CC79A7",  # purple
    "#000000",  # black
]

In [ ]:
def get_company_color(company):
    if company == 6:
        return sns.color_palette("Set2")[company - 1 % len(sns.color_palette("Set2"))]
    return sns.color_palette("Set1")[company - 1 % len(sns.color_palette("Set1"))]

In [8]:
# Criação de grafo direcionado: cada nó é um trajectory_id, arestas apontam de "seguidores" para "líderes" em cada instante

G = nx.DiGraph()

# Para cada instante de tempo, identificar quem está à frente (maior valor de 'y') em cada grupo (Company/Platoon)
for (company, platoon, t), group in enriched_points.groupby(['Company', 'Platoon', 't']):
    if len(group) < 2:
        continue
    # Ordena por 'y' decrescente (quem está mais à frente)
    sorted_group = group.sort_values('y', ascending=False)
    leader_id = sorted_group.iloc[0]['trajectory_id']
    for _, row in sorted_group.iloc[1:].iterrows():
        follower_id = row['trajectory_id']
        # Adiciona aresta do seguidor para o líder
        G.add_edge(follower_id, leader_id)

# Algoritmos de centralidade
pagerank = nx.pagerank(G)
degree_centrality = nx.degree_centrality(G)

# Exibir top 5 líderes por PageRank e Degree Centrality
top_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:5]
top_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]

print("Top 5 líderes por PageRank:", top_pagerank)
print("Top 5 líderes por Degree Centrality:", top_degree)

Top 5 líderes por PageRank: [(np.float64(107.0), 0.020806794055201697), (101.0, 0.020806794055201697), (100.0, 0.020806794055201697), (50.0, 0.01178343949044586), (np.float64(49.0), 0.01178343949044586)]
Top 5 líderes por Degree Centrality: [(35.0, 0.14102564102564102), (np.float64(36.0), 0.14102564102564102), (34.0, 0.14102564102564102), (33.0, 0.14102564102564102), (32.0, 0.14102564102564102)]
